In [12]:
from .finder import extend_file_finder

import builtins
import sys
import traceback
from pathlib import Path

Here we implement a no-op implementation of the loader interface:

In [16]:
def noop_loader(fullname, path):
    return None

We want to lazily load the notebook loader factory so that the Python startup is not adversely affected by importing nbconvert et al. However, if we do this naively with a relative import that is evaluated per-notebook, we will run into import loops. Here we'll define a lazy proxy that ensures the import hook is fully loaded.

In [21]:
def notebook_loader_factory(fullname, path):
    try:
        factory = notebook_loader_factory.factory
    except AttributeError:
        # As we're installing the notebook hook, we shouldn't be loading any notebooks yet
        notebook_loader_factory.factory = noop_loader

        # Realise import of loader
        from .importer import get_loader
        notebook_loader_factory.factory = get_loader

        factory = get_loader

    return factory(fullname, path)

Now we can install the import hook with a simple file finder

In [4]:
def install_import_hook(set_except_hook=True):
    extend_file_finder((notebook_loader_factory, [".ipynb"]),)

    if set_except_hook:
        sys.excepthook = traceback.print_exception